In [ ]:
# === SECTION 1: SHAP FEATURE IMPORTANCE GRAPHS===

import shap
import numpy as np
import pandas as pd

# Use your trained MLP as the winner model
winner_model = model_mlp

# ✅ Use the IQR-filtered + scaled datasets from your previous cells
background = X_Train.sample(100, random_state=42)
X_shap = X_Test.sample(100, random_state=42)

# Wrap predict_proba so it returns ONLY the probability for class 1
def mlp_prob_class1(x):
    # SHAP may pass numpy arrays; convert back to DataFrame with correct columns
    if isinstance(x, np.ndarray):
        x = pd.DataFrame(x, columns=background.columns)
    return winner_model.predict_proba(x)[:, 1]

# KernelExplainer for tabular NN
explainer = shap.KernelExplainer(mlp_prob_class1, background)

# Compute SHAP values
shap_values = explainer.shap_values(X_shap)

# Handle possible list output
sv = shap_values[0] if isinstance(shap_values, list) else shap_values

# ----------------------------
# ✅ MOST IMPORTANT FEATURES
# ----------------------------
importance = np.abs(sv).mean(axis=0)  # mean(|SHAP|) per feature
feat_imp = pd.Series(importance, index=X_shap.columns).sort_values(ascending=False)

top_n = 5
print(f"\nTop {top_n} Most Important Features (mean(|SHAP|)):\n")
print(feat_imp.head(top_n))

# Optional: bar chart ranking (global importance)
shap.summary_plot(sv, X_shap, plot_type="bar", feature_names=X_shap.columns, color="#FF6F61", show=True)

# Your original beeswarm summary plot (like the graph you showed)
shap.summary_plot(
    sv,
    X_shap,
    feature_names=X_shap.columns,
    cmap="viridis",
    show=True
)

In [ ]:
# === SECTION 2:SHAP Decision Plot for Heart Disease PRESENCE & ABSENCE ===

import matplotlib.pyplot as plt

# 1. Get predictions for the specific sample set used in SHAP
#    We need to know which rows correspond to "High Risk" and "Low Risk"
probs_shap = winner_model.predict_proba(X_shap)[:, 1]

# 2. Identify indices for the two cases
# Case A: Heart Disease Presence (Find the patient with Highest Probability)
idx_presence = np.argmax(probs_shap)

# Case B: Heart Disease Absence (Find the patient with Lowest Probability)
idx_absence = np.argmin(probs_shap)

# 3. Handle Expected Value (Base Value)
# KernelExplainer sometimes wraps expected_value in a list/array
base_value = explainer.expected_value
if isinstance(base_value, (list, np.ndarray)):
    base_value = base_value[0]

print(f"Base Value (Average Prediction): {base_value:.4f}")
print(f"Plotting Case 1 (Presence): Index {idx_presence}, Prob: {probs_shap[idx_presence]:.4f}")
print(f"Plotting Case 2 (Absence):  Index {idx_absence}, Prob: {probs_shap[idx_absence]:.4f}")

# ---------------------------------------------------------
# GRAPH 1: SHAP Decision Plot for Heart Disease PRESENCE
# ---------------------------------------------------------
print("\n--- 1. SHAP Decision Plot: Heart Disease Presence ---")
plt.figure()  # Create a new figure
shap.decision_plot(
    base_value,
    sv[idx_presence],
    X_shap.iloc[idx_presence],
    feature_names=X_shap.columns.tolist(),
    link='logit',  # Optional: converts log-odds to probabilities (0-1 scale)
    show=False     # Keep false to allow title addition
)
plt.title(f"Decision Plot: Heart Disease Presence (Prob: {probs_shap[idx_presence]:.2f})", fontsize=12)
plt.show()

# ---------------------------------------------------------
# GRAPH 2: SHAP Decision Plot for Heart Disease ABSENCE
# ---------------------------------------------------------
print("\n--- 2. SHAP Decision Plot: Heart Disease Absence ---")
plt.figure()  # Create a new figure
shap.decision_plot(
    base_value,
    sv[idx_absence],
    X_shap.iloc[idx_absence],
    feature_names=X_shap.columns.tolist(),
    link='logit',
    show=False
)
plt.title(f"Decision Plot: Heart Disease Absence (Prob: {probs_shap[idx_absence]:.2f})", fontsize=12)
plt.show()

In [ ]:
# === SECTION 3: LIME INSTANCE EXPLANATION ===
# For Patient-15

import lime
import lime.lime_tabular
import numpy as np

# 1. Setup the Explainer
# LIME needs to know the distribution of your training data to make "fake" perturbations
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_Train),      # Your training data
    feature_names=X_Train.columns,  # Name of your columns (Age, Sex_M, etc.)
    class_names=['Normal', 'Heart Disease'], # The labels 0 and 1
    mode='classification'
)

# 2. Pick a Specific Patient to Explain
# Let's pick patient #10 from the Test Set (you can change this number)
patient_idx = 15
patient_data = X_Test.iloc[patient_idx]
true_status = "Heart Disease" if Y_Test.iloc[patient_idx] == 1 else "Normal"

print(f"--- Explaining Patient #{patient_idx} ---")
print(f"Actual Truth: {true_status}")

# 3. Generate the Explanation
# We ask the MLP model: "What is the probability for this patient?"
exp = explainer.explain_instance(
    data_row=patient_data,
    predict_fn=model_mlp.predict_proba # We pass the MLP's prediction function
)

# 4. Show the Plot
exp.show_in_notebook(show_table=True)

# Optional: Save as HTML file to include in your paper presentation
exp.save_to_file('LIME_Explanation_Patient_15.html')

import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Extract data
explanation_data = exp.as_list()
feature_names = [x[0] for x in explanation_data]
feature_vals = [x[1] for x in explanation_data]

# --- THIS IS THE LINE TO EDIT FOR COLORS ---
# Currently set to Soft Coral (#FF6F61) and Pastel Teal (#48C9B0)
colors = ['#FF6F61' if x > 0 else '#48C9B0' for x in feature_vals]
# -------------------------------------------

# Create Plot
plt.figure(figsize=(10, 6))
plt.barh(feature_names, feature_vals, color=colors, edgecolor='white', height=0.7)

# Styling
plt.title(f"LIME Analysis: Patient #{patient_idx} (True: {true_status})", fontsize=14, fontweight='bold', pad=20)
plt.xlabel("Contribution to Prediction", fontsize=12)
plt.axvline(0, color='grey', linewidth=0.8, linestyle='--')
plt.grid(axis='x', linestyle=':', alpha=0.6)

# Legend
legend_elements = [
    Patch(facecolor='#FF6F61', label='Increases Risk(Heart Disease)'),
    Patch(facecolor='#48C9B0', label='Decreases Risk(Normal)')
]
plt.legend(handles=legend_elements, loc='lower right')

plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()